In [ ]:
!pip install hyperas

In [ ]:
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def prepare_data():
    """ 
    準備資料
    """
    # 欲使用的函式庫
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import KFold
    ## keras modules
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
    from tensorflow.keras.layers import Conv2D, MaxPooling2D # convolution layers

    # 從train.csv讀入pandas的DataFrame
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv') 
    train_x = train.drop(['label'], axis=1)   # 從train取出圖像資料 
    train_y = train['label']                  # 從train取出正確答案
    test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
     # 將train的資料分為訓練資料與驗證資料。
    kf = KFold(n_splits=4, shuffle=True, random_state=123)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 將圖像的像素值除以255.0，限制在0 ~ 1.0的範圍內，並轉換為numpy.array
    tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

    # 將二維矩陣的圖像數據變換成(高度 = 28, 寬度 = 28, 通道 = 1)的三維矩陣。
    # 灰階圖片的通道數為1。
    tr_x = tr_x.reshape(-1,28,28,1)
    va_x = va_x.reshape(-1,28,28,1)

    # 將正確答案轉換為One-hot encoding呈現
    tr_y = to_categorical(tr_y, 10)
    va_y = to_categorical(va_y, 10)
    return tr_x, tr_y, va_x, va_y  

def create_model(tr_x, tr_y): 
    """ 
    建立模型
    """ 
    # 建立Sequential物件 
    model = Sequential()  
    # 探索第1層的過濾器數量、過濾器尺寸
    model.add(Conv2D(filters={{choice([32, 64])}},
                     kernel_size={{choice([(3,3), (5,5), (7,7)])}},
                     padding='same',
                     activation={{choice(['tanh', 'relu'])}},
                     input_shape=(28,28,1)))

    # 探索第2層的過濾器數量、過濾器尺寸
    model.add(Conv2D(filters = {{choice([32, 64])}},
                     kernel_size = {{choice([(3,3), (5,5), (7,7)])}},
                     padding='same',
                     activation={{choice(['tanh', 'relu'])}}))

    # 在第3層配置2×2的池化層
    model.add(MaxPooling2D(pool_size=(2,2)))

    # 探索丟棄率。
    model.add(Dropout({{quniform(0.2, 0.6, 0.05)}}))

    # 探索第4層的過濾器數量、過濾器尺寸
    model.add(Conv2D(filters={{choice([32, 64])}},
                     kernel_size={{choice([(3,3), (5,5), (7,7)])}},
                     padding='same',
                     activation='relu'))

    # 探索第5層的過濾器數量、過濾器尺寸
    model.add(Conv2D(filters = {{choice([32, 64])}},
                     kernel_size = {{choice([(3,3), (5,5), (7,7)])}},
                     padding='same',
                     activation={{choice(['tanh', 'relu'])}} )) 
    
    # 在第6層配置2×2的池化層 
    model.add(MaxPooling2D(pool_size=(2,2)))  
    
    # 探索丟棄率
    model.add(Dropout({{quniform(0.2, 0.6, 0.05)}}))

    # 配置扁平層
    model.add(Flatten())

    # 從1,2裡面探索要追加的層的數量
    if {{choice(['one', 'two'])}} == 'one':
        """ 
        選到one的時候就配置第7層，並探索神經元數量、激活函數、丟棄率
        """
        # 第7層
        model.add(Dense({{choice([500, 600, 700])}},
                        activation={{choice(['tanh', 'relu'])}}))
        model.add(Dropout({{quniform(0.1, 0.6, 0.05)}}))
        
    elif {{choice(['one', 'two'])}} == 'two':
        """
        選到two的時候就配置第7層跟第8層，並探索神經元數量、激活函數、丟棄率
        """

        # 第7層
        model.add(Dense({{choice([500, 600, 700])}},
                        activation={{choice(['tanh', 'relu'])}}))
        model.add(Dropout({{quniform(0.1, 0.6, 0.05)}}))
        
        # 第8層
        model.add(Dense({{choice([100, 150, 200])}},
                        activation={{choice(['tanh', 'relu'])}}))
        model.add(Dropout({{quniform(0.2, 0.6, 0.05)}}))
    
    # 放置輸出層
    model.add(Dense(10, activation = "softmax"))

    # 模型編譯
    # 嘗試使用Adam跟RMSprop作為優化器
    model.compile(loss="categorical_crossentropy",
                  optimizer={{choice(['adam', 'rmsprop'])}},
                  metrics=["accuracy"])

    # 訓練次數為30次
    epoch = 30

    # 用100與200作為批次大小。
    batch_size = {{choice([100, 200, 300])}}
    result = model.fit(tr_x, tr_y,
                       epochs=epoch,
                       batch_size=batch_size,
                       validation_data=(va_x, va_y),
                       verbose=0)

    # 簡單輸出訓練時的結果
    validation_acc = np.amax(result.history['val_accuracy'])
    print('Best validation acc of epoch:', validation_acc)

    # 探索如何能讓validation_acc數值最小化
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

# 設定執行75次的探索
best_run, best_model = optim.minimize(model=create_model,
                                      data=prepare_data,
                                      algo=tpe.suggest,
                                      max_evals=75,
                                      eval_space=True,
                                      notebook_name='__notebook_source__',
                                      trials=Trials())


In [ ]:
# 輸出準確度最高的模型
print(best_model.summary()) 
# 輸出準確度最好的參數數值
print(best_run)

# 使用驗證資料檢驗已完成探索的模型。
_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss: ", val_loss)    # 輸出損失
print("val_acc: ", val_acc)      # 輸出準確度
